<h1 align="center">Lab 2:  Sexism Identification in Twitter</h1>
<h2 align="center">Session 1. Machine Learning and Feature Engineering</h2>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Marcos Ranchal
- Marc Siquier

In [180]:
# Reading the entire dataset for both languages and considering only the hard labels. In this lab we do not address the sexism identification task from a Learning with Disagreement (LwD) perspective.

from readerEXIST2025 import EXISTReader

reader_train = EXISTReader("EXIST_2025_Dataset_V0.2/EXIST2025_training.json")
reader_dev = EXISTReader("EXIST_2025_Dataset_V0.2/EXIST2025_dev.json")

EnTrainTask1, EnDevTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2")

SpTrainTask1, SpDevTask1 = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2 = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2")


Voy a ir explicando el proceso paso a paso para que puedas replicarlo, luego cuando lo tengamos lo arreglamos y traducimos y ponemos alguna conclusión final o algo

# ENGLISH

## Preprocessing

In [181]:
# COMPLETE IF YOU WANT TO DO TEXT PREPROCESSING
import nltk
import re
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import emoji

web_re = re.compile(r"https?:\/\/[^\s]+", re.U)
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)
emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
pattern_emoji = '(' + '|'.join(re.escape(u) for u in emojis) + ')'
symbols=r"[()'\"¿?!¡;,:.?\.]"
decimal = r"(\d+[.,]\d+(?:[%]?)?)"
dates = r"(\d{1,2}[/\-]\d{1,2}(?:[/\-]\d{2,4})?)"
hora = r"\b\d{1,2}:\d{2}(?::\d{2})?\s?(?:[ap]\.?m\.?)?"
fechas_format = r"(\d{1,2}\s+\w+\s+\d{4})"


stopw = {
    "english": nltk.corpus.stopwords.words("english"),
    "spanish": nltk.corpus.stopwords.words("spanish")
}


def preprocess(text):
    # COMPLETE
    text = web_re.sub("", text)
    text = user_re.sub("", text)
    text = hashtag_re.sub("", text)
    text = re.sub(pattern_emoji, "", text)
    text = re.sub(symbols, "", text)
    text = re.sub(dates, "", text)
    text  = re.sub(decimal, "", text)
    text  = re.sub(hora, "", text)
    text = re.sub(fechas_format, "", text)
    text = text.lower()
    return text

def tokenize(text_list, lang="english"):
    # COMPLETE
    token_list = []
    for t in text_list:
        text = preprocess(t)
        list_t = nltk.tokenize.word_tokenize(text, language=lang, preserve_line=False)
        list_t = [word for word in list_t if word not in stopw[lang]]
        token_list.append(" ".join(list_t))
    return token_list

tokenized_text = {
    "english_train_1": tokenize(EnTrainTask1[1], "english"),
    "english_test_1": tokenize(EnDevTask1[1], "english"),
    "english_train_2": tokenize(EnTrainTask2[1], "english"),
    "english_test_2": tokenize(EnDevTask2[1], "english")
}



Primero, aplicamos limpieza de texto utilizando los patrones re de la sesión 1 del laboratorio, para eliminar elementos extraños.

Además, detectamos los stop words con nltk y los eliminamos de cada tweet para que no afecten.

Vemos el resultado:

In [182]:
for i in range(min(5, len(tokenized_text["english_train_1"]))):
    print(tokenized_text["english_train_1"][i])


writing uni essay local pub coffee random old man keeps asking drunk questions im trying concentrate & amp ends good luck youll end getting married use anyway alive well
20 dont appreciate two rides team member looked behind asked man behind many party impressed
according customer plenty time go spent stirling coins wants pay derry like woman im sure retail
blokes drink beer sorry arent bloke drink wine apparently alive well
new shelves week - looking forward reading books


## Tweet representations (Feature extraction)

Aquí se hace lo siguiente:
1. Convertir la columna 2 de los datasets (etiquetas) a listas.
2. Dividir cada train con train_test_split (no sería necesario realmente).
3. Se carga un SVD (n=100) para reducir la dimensionalidad, un vectorizer basado en Term Frequency y un Label Encoder para codificar las etiquetas (one hot encoding).
4. Se convierten los vectores de entrenamiento y de test (primero pasan por el vectorizer, y luego por el SVD para reducir las dimensiones).
5. A las Y's se le aplica el Label Encoder.

NOTA: Es importante que se utilicen instancias diferentes de cada modelo para las transformaciones en la task1 y en la task2, pues task1 es binaria y task2 multiclase (si no da errores).

Esta representación es estática.

In [183]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.svm import SVC
import xgboost as xgb
import numpy as np 




y = EnTrainTask1[2].tolist()
X_train, y_train = tokenized_text["english_train_1"], y


svd1 = TruncatedSVD(n_components=100)

vectorizer_task1 = TfidfVectorizer(norm="l2")
label_encoder_task1 = LabelEncoder()

X_train_vectors = vectorizer_task1.fit_transform(X_train)
lsa_vectors_train_1 = svd1.fit_transform(X_train_vectors)



y_train_numeric = label_encoder_task1.fit_transform(y_train)


y2 = EnTrainTask2[2].tolist()
X_train_2,  y_train_2 = tokenized_text["english_train_2"], y2

vectorizer_task2 = TfidfVectorizer(norm="l2")
label_encoder_task2 = LabelEncoder()
svd2 = TruncatedSVD(n_components=100)

X_train_vectors_2 = vectorizer_task2.fit_transform(X_train_2)
lsa_vectors_train_2 = svd2.fit_transform(X_train_vectors_2)



y_train_numeric_2 = label_encoder_task2.fit_transform(y_train_2)


Verificamos las dimensiones de los lsa_vectors, en todo caso deben tener 100 columnas cada uno por la reducción de dimensionalidad.

In [184]:
print("Dimensiones de lsa_vectors_train_1:", lsa_vectors_train_1.shape)
print("Dimensiones de lsa_vectors_train_2:", lsa_vectors_train_2.shape)


Dimensiones de lsa_vectors_train_1: (2870, 100)
Dimensiones de lsa_vectors_train_2: (856, 100)


Aquí se aplican las mismas transformaciones que se han hecho sobre los datos de entrenamiento sobre los datos de test (Dev). Hay que asegurarse de que los vectorizers, SVD y label encoders son los mismos que antes en su respectiva tarea.

In [185]:
real_test_X_task1 = tokenized_text["english_test_1"]
X_test_vectors_task1 = vectorizer_task1.fit_transform(real_test_X_task1)
lsa_vectors_test_task1 = svd1.fit_transform(X_test_vectors_task1)

real_test_y_task1 = EnDevTask1[2].tolist()
y_test_task1 = label_encoder_task1.fit_transform(real_test_y_task1)



real_test_X_task2 = tokenized_text["english_test_2"]
X_test_vectors_task2 = vectorizer_task2.fit_transform(real_test_X_task2)
lsa_vectors_test_task2 = svd2.fit_transform(X_test_vectors_task2)

real_test_y_task2 = EnDevTask2[2].tolist()
y_test_task2 = label_encoder_task2.fit_transform(real_test_y_task2)


Comprobamos nuevamente la dimensionalidad, pero ahora de los vectores de test.

In [186]:
print(lsa_vectors_test_task1.shape)
print(lsa_vectors_test_task2.shape)

(444, 100)
(146, 100)


A continuación se obtienen los embeddings contextuales con BERT.

In [187]:

import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


if torch.backends.mps.is_available():  # Mac M? GPU
    device = torch.device("mps")
elif torch.cuda.is_available():  # Nvidia GPU
    device = torch.device("cuda")
else:  # CPU
    device = torch.device("cpu")



modelnames = {"english": ["roberta-base"], "spanish": ["PlanTL-GOB-ES/roberta-base-bne"]}   

tokenizers = {"english": [], "spanish": []}
models = {"english": [], "spanish": []}

for lang in ["english", "spanish"]:
    for modelname in modelnames[lang]:
            tokenizer = RobertaTokenizer.from_pretrained(modelname)
            model = RobertaModel.from_pretrained(modelname)
            tokenizers[lang].append(tokenizer)
            models[lang].append(model)
        
batch_size = 16 

# COMPLETE


vectors = {"task1": {"english": {"train":{}, "test":{}}, "spanish": {"train":{}, "test":{}}}, "task2":{"english": {"train":{}, "test":{}}, "spanish": {"train":{}, "test":{}}}}


tweetsTotal = [tokenized_text["english_train_1"], tokenized_text["english_train_2"], tokenized_text["english_test_1"], tokenized_text["english_test_2"]]

task = 1

lang = "english"

for tweets in tweetsTotal:
    for i, model in enumerate(models[lang]):
        tokenizer = tokenizers[lang][i]
        model = models[lang][i]
        name = modelnames[lang][i]


        tensor_list=[]


        for i in range(0, len(tweets), batch_size):
            batch = tweets[i:i+batch_size]
            input = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            model.eval()
            model.to(device)
            input = input.to(device)
            with torch.no_grad():
                outputs = model(**input)
                # Transformers models always return tuples.
                # Here, the first element corresponds to the vectors in the output of the last BETO layer.
                encoded_layers = outputs[0]
                #Here we obtain the embedding of the CLS tokens for each input text.
                #This representation serves as a contextual embedding of the texts.
                cls_vector = encoded_layers[:,0,:]
                #Vector associated with the CLS token of the first text in the entry.
            tensor_list.append(cls_vector)
        cls_vector = torch.cat(tensor_list).cpu()#.detach().numpy()[0]
        if task == 1:
            vectors["task1"][lang]["train"][name] = cls_vector
        elif task == 2:
            vectors["task2"][lang]["train"][name] = cls_vector
        elif task == 3:
            vectors["task1"][lang]["test"][name] = cls_vector
        else:
            vectors["task2"][lang]["test"][name] = cls_vector

        task += 1
        print(f"Model: {name}, {(cls_vector.size())}")





Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: roberta-base, torch.Size([2870, 768])
Model: roberta-base, torch.Size([856, 768])
Model: roberta-base, torch.Size([444, 768])
Model: roberta-base, torch.Size([146, 768])


Mostramos un ejemplo de los vectores de entrenamiento para ver que son tensores de las dimensiones adecuadas

In [188]:
vectors["task1"]["english"]["train"]["roberta-base"]

tensor([[-0.0444,  0.0395, -0.0468,  ..., -0.1237, -0.0360,  0.0144],
        [-0.0409,  0.0366, -0.0349,  ..., -0.0743, -0.0789, -0.0020],
        [-0.0126,  0.0489, -0.0073,  ..., -0.1055, -0.0543, -0.0028],
        ...,
        [-0.0002,  0.0236, -0.0423,  ..., -0.1465, -0.0481,  0.0078],
        [-0.0559,  0.0523, -0.0239,  ..., -0.1038, -0.0295, -0.0287],
        [-0.0385,  0.0478, -0.0233,  ..., -0.1351, -0.0708, -0.0144]])

## Learning Models

A continuación se entrenan los modelos.

1. MLP
2. XGBoost

Se usan los vectores de train, lsa_vectors_train, en el caso de static embeddings, y vectors["task1"]["english"]["train"]["roberta-base"], en el caso de contextual embeddings. Se consiguen al final un total de 2 (modelos) x 2 (métodos de embeddings) x 2 (tasks) = 8 predicciones finales.



Modelos tarea 1:

In [189]:
### STATIC

clf1_en_1_static = SVC(random_state=1, max_iter=1000)
clf1_en_1_static.fit(lsa_vectors_train_1, y_train_numeric)

# Convert data to DMatrix format
dtrain1_en_static = xgb.DMatrix(lsa_vectors_train_1, label=y_train_numeric)
dtest1_en_static = xgb.DMatrix(lsa_vectors_test_task1, label=y_test_task1)
# Set XGBoost parameters
params_static = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}
# Train the model
num_rounds_static = 100
bst_en_1_static = xgb.train(params_static, dtrain1_en_static, num_rounds_static)



### CONTEXTUAL

clf1_en_1_contextual = SVC(random_state=1, max_iter=1000)
clf1_en_1_contextual.fit(vectors["task1"]["english"]["train"]["roberta-base"], y_train_numeric)


# Convert data to DMatrix format
dtrain1_en_contextual = xgb.DMatrix(vectors["task1"]["english"]["train"]["roberta-base"], label=y_train_numeric)
dtest1_en_contextual = xgb.DMatrix(vectors["task1"]["english"]["test"]["roberta-base"], label=y_test_task1)
# Set XGBoost parameters
params_contextual = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}
# Train the model
num_rounds_contextual = 100
bst_en_1_contextual = xgb.train(params_contextual, dtrain1_en_contextual, num_rounds_contextual)



C:\Users\marcos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\marcos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Modelos tarea 2:

In [190]:
clf1_en_2_static = SVC(random_state=1, max_iter=1000)
clf1_en_2_static.fit(lsa_vectors_train_2, y_train_numeric_2)

# Convert data to DMatrix format
dtrain2_en_static = xgb.DMatrix(lsa_vectors_train_2, label=y_train_numeric_2)
dtest2_en_static = xgb.DMatrix(lsa_vectors_test_task2, label=y_test_task2)
# Set XGBoost parameters
params_static = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',  # Change objective to multi:softmax for multi-class classification
    'num_class': 3,  # Specify the number of classes
    'eval_metric': 'mlogloss'
}
# Train the model
num_rounds_static = 100
bst_en_2_static = xgb.train(params_static, dtrain2_en_static, num_rounds_static)



### CONTEXTUAL

clf1_en_2_contextual = SVC(random_state=1, max_iter=1000)
clf1_en_2_contextual.fit(vectors["task2"]["english"]["train"]["roberta-base"], y_train_numeric_2)

# Convert data to DMatrix format
dtrain2_en_contextual = xgb.DMatrix(vectors["task2"]["english"]["train"]["roberta-base"], label=y_train_numeric_2)
dtest2_en_contextual = xgb.DMatrix(vectors["task2"]["english"]["test"]["roberta-base"], label=y_test_task2)
# Set XGBoost parameters
params_contextual = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',  # Change objective to multi:softmax for multi-class classification
    'num_class': 3,  # Specify the number of classes
    'eval_metric': 'mlogloss'
}
# Train the model
num_rounds_contextual = 100
bst_en_2_contextual = xgb.train(params_contextual, dtrain2_en_contextual, num_rounds_contextual)




Finalmente realizamos predict sobre cada modelo con los lsa_vectors_test_taskX, que se corresponden con los vectores del dataset de test (Dev). Volvemos a generar la matriz XGB para el modelo XGBoost utilizando los datos Dev.

In [191]:

predicted1_1_static = clf1_en_1_static.predict(lsa_vectors_test_task1)

predicted1_1_contextual = clf1_en_1_contextual.predict(vectors["task1"]["english"]["test"]["roberta-base"])

# Make predictions on the test set
predicted1_2_static = bst_en_1_static.predict(dtest1_en_static)
predicted1_2_static = np.asarray([int(round(value)) for value in predicted1_2_static])

# Make predictions on the test set
predicted1_2_contextual = bst_en_1_contextual.predict(dtest1_en_contextual)
predicted1_2_contextual = np.asarray([int(round(value)) for value in predicted1_2_contextual])

# Generar predicciones para EnDevTask2 usando los modelos _2
dtest2_en = xgb.DMatrix(lsa_vectors_test_task2, label=y_test_task2)

predicted2_1_static = clf1_en_2_static.predict(lsa_vectors_test_task2)

predicted2_1_contextual = clf1_en_2_contextual.predict(vectors["task2"]["english"]["test"]["roberta-base"])

# Make predictions on the test set
predicted2_2_static = bst_en_2_static.predict(dtest2_en_static)
predicted2_2_static = np.asarray([int(round(value)) for value in predicted2_2_static])

# Make predictions on the test set
predicted2_2_contextual = bst_en_2_contextual.predict(dtest2_en_contextual)
predicted2_2_contextual = np.asarray([int(round(value)) for value in predicted2_2_contextual])


## Show Results

Mostramos los resultados de cada modelo para cada tarea. Utilizamos la función del profesor show_subtaskX, que recibe de argumentos ref (y del dataset Dev de la tarea) y pred (las predicciones finales de nuestros modelos a partir de las X del dataset Dev de la tarea).

In [192]:
#EXAMPLE 1 TASK1 and TASK2 FOR ENGLISH
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

def show_subtask1(ref, pred):
    # Computing the F1 Score for the positive class (sexism), subtask 1
    f1_positive = f1_score(ref, pred, pos_label=1, zero_division=0)
    # Obtainig a detailed classification report
    report = classification_report(ref, pred, digits=4, zero_division=0)   
    print("\nSUBTASK 1")
    print(f"F1-score (Positive Class): {f1_positive}")
    print(report)

def show_subtask2(ref, pred):
    # Computing the macro-average F1 for the second subtask
    f1_macro = f1_score(ref, pred, average='macro', zero_division=0)
    # Obtainig a detailed classification report
    report = classification_report(ref,pred, digits=4, zero_division=0)
    print("\nSUBTASK 2")
    print(f"F1-score (Macro-Averaged): {f1_macro}")
    print(report)


###########################SUBTASK 1##################################

# COMPLETE
print("ENGLISH\n")
print("STATIC EMBEDDINGS: \n")
# Subtask 1
print("SUBTASK 1\n")
print("=================================\n")
print("MLP: ")
show_subtask1(y_test_task1, predicted1_1_static)

print("=================================\n")
print("XGBoost: ")
show_subtask1(y_test_task1, predicted1_2_static)
print("=================================\n")

# Subtask 2
print("SUBTASK 2\n")
print("=================================\n")
print("MLP: ")
show_subtask2(y_test_task2, predicted2_1_static)
print("=================================\n")
print("XGBoost: ")
show_subtask2(y_test_task2, predicted2_2_static)
print("=================================\n")


print("ENGLISH\n")
print("CONTEXTUAL EMBEDDINGS: \n")
# Subtask 1
print("SUBTASK 1\n")
print("=================================\n")
print("MLP: ")
show_subtask1(y_test_task1, predicted1_1_contextual)
print("=================================\n")
print("XGBoost: ")
show_subtask1(y_test_task1, predicted1_2_contextual)
print("=================================\n")

# Subtask 2
print("SUBTASK 2\n")
print("=================================\n")
print("MLP: ")
show_subtask2(y_test_task2, predicted2_1_contextual)
print("=================================\n")
print("XGBoost: ")
show_subtask2(y_test_task2, predicted2_2_contextual)
print("=================================\n")


ENGLISH

STATIC EMBEDDINGS: 

SUBTASK 1


MLP: 

SUBTASK 1
F1-score (Positive Class): 0.5876288659793815
              precision    recall  f1-score   support

           0     0.6800    0.6800    0.6800       250
           1     0.5876    0.5876    0.5876       194

    accuracy                         0.6396       444
   macro avg     0.6338    0.6338    0.6338       444
weighted avg     0.6396    0.6396    0.6396       444


XGBoost: 

SUBTASK 1
F1-score (Positive Class): 0.6142131979695431
              precision    recall  f1-score   support

           0     0.7008    0.6840    0.6923       250
           1     0.6050    0.6237    0.6142       194

    accuracy                         0.6577       444
   macro avg     0.6529    0.6539    0.6533       444
weighted avg     0.6590    0.6577    0.6582       444


SUBTASK 2


MLP: 

SUBTASK 2
F1-score (Macro-Averaged): 0.24531024531024528
              precision    recall  f1-score   support

           0     0.5822    1.0000    0.73

# SPANISH

Se utiliza el mismo procedimiento descrito anteriormente dado que el único cambio realizado es el idioma empleado. Esto implica mantener los métodos de evaluación, métricas y modelos empleados previamente, aplicando exactamente los mismos pasos para asegurar la comparabilidad directa de los resultados.

## Preprocessing

In [206]:
import re
import nltk

# FUNCIONES PARA PREPROCESAMIENTO DE TEXTO
# Limpieza del texto en español
def limpiar_texto_esp(texto):
    texto = web_re.sub("", texto)
    texto = user_re.sub("", texto)
    texto = hashtag_re.sub("", texto)
    texto = re.sub(pattern_emoji, "", texto)
    texto = re.sub(symbols, "", texto)
    texto = re.sub(dates, "", texto)
    texto = re.sub(decimal, "", texto)
    texto = re.sub(hora, "", texto)
    texto = re.sub(fechas_format, "", texto)
    return texto.lower()

# Tokenización de textos en español
def tokenizar_esp(lista_textos):
    lista_tokens = []
    for texto in lista_textos:
        texto_procesado = limpiar_texto_esp(texto)
        tokens = nltk.tokenize.word_tokenize(texto_procesado, language="spanish", preserve_line=False)
        tokens_filtrados = [palabra for palabra in tokens if palabra not in stopw["spanish"]]
        lista_tokens.append(" ".join(tokens_filtrados))
    return lista_tokens

# Tokenización de los conjuntos de entrenamiento y prueba
tokenized_text["spanish_train_1"] = tokenizar_esp(SpTrainTask1[1])
tokenized_text["spanish_test_1"] = tokenizar_esp(SpDevTask1[1])
tokenized_text["spanish_train_2"] = tokenizar_esp(SpTrainTask2[1])
tokenized_text["spanish_test_2"] = tokenizar_esp(SpDevTask2[1])

tokenized_text["spanish_train_1"] = tokenize(SpTrainTask1[1], "spanish")
tokenized_text["spanish_test_1"] = tokenize(SpDevTask1[1], "spanish")
tokenized_text["spanish_train_2"] = tokenize(SpTrainTask2[1], "spanish")
tokenized_text["spanish_test_2"] = tokenize(SpDevTask2[1], "spanish")

# Mostrar ejemplos de los textos tokenizados
for idx in range(min(5, len(tokenized_text["spanish_train_1"]))):
    print(tokenized_text["spanish_train_1"][idx])

ignora capulloel problema youtuber denuncia acoso afecta gente izquierdas ejemplo video gamergate presenta normal acoso reciben fisher anita zöey amenazas bomba
si comicsgate parece gamergate pues bien acoso si haciendo sabotaje personajes gustan entonces gracias darme razón lloricas ofendidos
lee gamergate cambiado manera comunicamos internet fanboys halo tóxicos fanboys comunidades/juegos querido coger pauta
entonces así mercado mejor hacer cambiarlo seguir alimentando machismo consumidores lugar apoyar gente víctimas gamergateacerca implica imperativo entonces entiendo buscaban
aaah andrew dobson dedicó echar mierda gamergate puro estilo white knight deviantart echó encima


In [207]:
# Transformaciones y vectorización de los datos
datos_prueba_X_tarea1 = tokenized_text["spanish_test_1"]
X_vectores_prueba_tarea1 = vectorizer_task1.fit_transform(datos_prueba_X_tarea1)
lsa_vectores_prueba_tarea1 = svd1.fit_transform(X_vectores_prueba_tarea1)

datos_prueba_y_tarea1 = SpDevTask1[2].tolist()
y_prueba_tarea1 = label_encoder_task1.fit_transform(datos_prueba_y_tarea1)

datos_prueba_X_tarea2 = tokenized_text["spanish_test_2"]
X_vectores_prueba_tarea2 = vectorizer_task2.fit_transform(datos_prueba_X_tarea2)
lsa_vectores_prueba_tarea2 = svd2.fit_transform(X_vectores_prueba_tarea2)

datos_prueba_y_tarea2 = SpDevTask2[2].tolist()
y_prueba_tarea2 = label_encoder_task2.fit_transform(datos_prueba_y_tarea2)

## Tweet representations (Feature extraction)

In [208]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.svm import SVC
import xgboost as xgb
import numpy as np 

# Obtención de representaciones para los subconjuntos de entrenamiento y desarrollo en ambas tareas
datos_y_tarea1 = SpTrainTask1[2].tolist()
datos_X_tarea1, etiquetas_y_tarea1 = tokenized_text["spanish_train_1"], datos_y_tarea1

reductor_tarea1 = TruncatedSVD(n_components=100)
vectorizador_tarea1 = TfidfVectorizer(norm="l2")
codificador_tarea1 = LabelEncoder()

X_vectores_entrenamiento1 = vectorizador_tarea1.fit_transform(datos_X_tarea1)
lsa_vectores_entrenamiento1 = reductor_tarea1.fit_transform(X_vectores_entrenamiento1)

y_etiquetas_numericas1 = codificador_tarea1.fit_transform(etiquetas_y_tarea1)

datos_y_tarea2 = SpTrainTask2[2].tolist()
datos_X_tarea2, etiquetas_y_tarea2 = tokenized_text["spanish_train_2"], datos_y_tarea2

vectorizador_tarea2 = TfidfVectorizer(norm="l2")
codificador_tarea2 = LabelEncoder()
reductor_tarea2 = TruncatedSVD(n_components=100)

X_vectores_entrenamiento2 = vectorizador_tarea2.fit_transform(datos_X_tarea2)
lsa_vectores_entrenamiento2 = reductor_tarea2.fit_transform(X_vectores_entrenamiento2)

y_etiquetas_numericas2 = codificador_tarea2.fit_transform(etiquetas_y_tarea2)


In [209]:
# Impresión de dimensiones de los vectores reducidos
print("Forma de lsa_vectores_entrenamiento1:", lsa_vectores_entrenamiento1.shape)
print("Forma de lsa_vectores_entrenamiento2:", lsa_vectores_entrenamiento2.shape)

Forma de lsa_vectores_entrenamiento1: (3194, 100)
Forma de lsa_vectores_entrenamiento2: (1217, 100)


In [210]:
# Transformación de datos de prueba
datos_prueba_X_tarea1 = tokenized_text["spanish_test_1"]
X_vectores_prueba_tarea1 = vectorizador_tarea1.transform(datos_prueba_X_tarea1)
lsa_vectores_prueba_tarea1 = reductor_tarea1.transform(X_vectores_prueba_tarea1)

datos_prueba_y_tarea1 = SpDevTask1[2].tolist()
y_prueba_tarea1 = codificador_tarea1.fit_transform(datos_prueba_y_tarea1)

datos_prueba_X_tarea2 = tokenized_text["spanish_test_2"]
X_vectores_prueba_tarea2 = vectorizador_tarea2.transform(datos_prueba_X_tarea2)
lsa_vectores_prueba_tarea2 = reductor_tarea2.transform(X_vectores_prueba_tarea2)

datos_prueba_y_tarea2 = SpDevTask2[2].tolist()
y_prueba_tarea2 = codificador_tarea2.fit_transform(datos_prueba_y_tarea2)


In [211]:
print(lsa_vectores_prueba_tarea1.shape)
print(lsa_vectores_prueba_tarea2.shape)

(490, 100)
(207, 100)


In [212]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

tam_lote = 16  

# COMPLETO
tweets_totales = [tokenized_text["spanish_train_1"], tokenized_text["spanish_train_2"], tokenized_text["spanish_test_1"], tokenized_text["spanish_test_2"]]

tarea = 1

idioma = "spanish"

for tweets in tweets_totales:
    for i, modelo in enumerate(models[idioma]):
        tokenizador = tokenizers[idioma][i]
        modelo = models[idioma][i]
        nombre = modelnames[idioma][i]

        lista_tensores = []

        for i in range(0, len(tweets), tam_lote):
            lote = tweets[i:i+tam_lote]
            entrada = tokenizador(lote, padding=True, truncation=True, return_tensors="pt")
            modelo.eval()
            modelo.to(device)
            entrada = entrada.to(device)
            with torch.no_grad():
                salidas = modelo(**entrada)
                capas_codificadas = salidas[0]
                vector_cls = capas_codificadas[:,0,:]
            lista_tensores.append(vector_cls)

        vector_cls = torch.cat(lista_tensores).cpu()

        if tarea == 1:
            vectors["task1"][idioma]["train"][nombre] = vector_cls
        elif tarea == 2:
            vectors["task2"][idioma]["train"][nombre] = vector_cls
        elif tarea == 3:
            vectors["task1"][idioma]["test"][nombre] = vector_cls
        else:
            vectors["task2"][idioma]["test"][nombre] = vector_cls

        tarea += 1
        print(f"Modelo: {nombre}, {vector_cls.size()}")  


Modelo: PlanTL-GOB-ES/roberta-base-bne, torch.Size([3194, 768])
Modelo: PlanTL-GOB-ES/roberta-base-bne, torch.Size([1217, 768])


KeyboardInterrupt: 

In [200]:
vectors["task2"]["spanish"]["train"]["PlanTL-GOB-ES/roberta-base-bne"]

tensor([[-0.0164, -0.0334,  0.0596,  ..., -0.0298, -0.0328,  0.1693],
        [-0.1075, -0.0646,  0.0651,  ..., -0.0653, -0.0259,  0.1355],
        [-0.1203, -0.1275,  0.0947,  ..., -0.0390,  0.0355,  0.1365],
        ...,
        [-0.1062, -0.0527,  0.0321,  ..., -0.0448,  0.0261,  0.1514],
        [-0.0356, -0.0579,  0.1164,  ..., -0.1082, -0.0406,  0.1134],
        [-0.0650, -0.0079,  0.0311,  ..., -0.0564,  0.0343,  0.1082]])

## Learning Models

In [ ]:
# COMPLETO  
### ESTÁTICO  

clasificador_estatico_1 = SVC(random_state=1, max_iter=1000)  
clasificador_estatico_1.fit(lsa_vectores_entrenamiento1, y_etiquetas_numericas1)  

# Convertir datos a formato DMatrix  
datos_entrenamiento_estatico_t1 = xgb.DMatrix(lsa_vectores_entrenamiento1, label=y_etiquetas_numericas1)  
datos_prueba_estatico_t1 = xgb.DMatrix(lsa_vectores_prueba_tarea1, label=y_prueba_tarea1)  

# Definir parámetros de XGBoost  
parametros_estatico = {  
    'max_depth': 3,  
    'eta': 0.1,  
    'objective': 'binary:logistic',  
    'eval_metric': 'auc'  
}  

# Entrenar el modelo  
rondas_estatico = 100  
modelo_estatico_1 = xgb.train(parametros_estatico, datos_entrenamiento_estatico_t1, rondas_estatico)  


### CONTEXTUAL  

clasificador_contextual_1 = SVC(random_state=1, max_iter=1000)  
clasificador_contextual_1.fit(vectors["task1"]["spanish"]["train"]["PlanTL-GOB-ES/roberta-base-bne"], y_etiquetas_numericas1)  

# Convertir datos a formato DMatrix  
datos_entrenamiento_contextual_t1 = xgb.DMatrix(vectors["task1"]["spanish"]["train"]["PlanTL-GOB-ES/roberta-base-bne"], label=y_etiquetas_numericas1)  
datos_prueba_contextual_t1 = xgb.DMatrix(vectors["task1"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"], label=y_prueba_tarea1)  

# Definir parámetros de XGBoost  
parametros_contextual = {  
    'max_depth': 3,  
    'eta': 0.1,  
    'objective': 'binary:logistic',  
    'eval_metric': 'auc'  
}  

# Entrenar el modelo  
rondas_contextual = 100  
modelo_contextual_1 = xgb.train(parametros_contextual, datos_entrenamiento_contextual_t1, rondas_contextual)  


C:\Users\marcos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\marcos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [ ]:
import xgboost as xgb
from sklearn.svm import SVC

# Entrenamiento del clasificador SVM usando vectores estáticos (e.g., LSA)
clasificador_estatico_2 = SVC(random_state=1, max_iter=1000)
clasificador_estatico_2.fit(
    lsa_vectores_entrenamiento2,  # Datos transformados con LSA para entrenamiento
    y_etiquetas_numericas2        # Etiquetas numéricas correspondientes a las clases
)

# Preparación de datos para entrenamiento y prueba con XGBoost utilizando vectores estáticos
# Convertimos los datos en formato DMatrix, formato eficiente para XGBoost
datos_entrenamiento_estatico = xgb.DMatrix(
    lsa_vectores_entrenamiento2,
    label=y_etiquetas_numericas2
)
datos_prueba_estatico = xgb.DMatrix(
    lsa_vectores_prueba_tarea2,
    label=y_prueba_tarea2
)

# Parámetros de entrenamiento para XGBoost (vectores estáticos)
params_estatico = {
    'max_depth': 3,              # Profundidad máxima de los árboles
    'eta': 0.1,                  # Tasa de aprendizaje
    'objective': 'multi:softmax',# Objetivo para clasificación multiclase
    'num_class': 3,              # Número de clases objetivo
    'eval_metric': 'mlogloss'    # Métrica de evaluación
}
num_rounds_estatico = 100        # Número de iteraciones del entrenamiento

# Entrenamiento del modelo XGBoost con datos estáticos
modelo_estatico_2 = xgb.train(
    params_estatico,
    datos_entrenamiento_estatico,
    num_rounds_estatico
)

# Entrenamiento del clasificador SVM usando embeddings contextuales (RoBERTa-BNE)
clasificador_contextual_2 = SVC(random_state=1, max_iter=1000)
clasificador_contextual_2.fit(
    vectors["task2"]["spanish"]["train"]["PlanTL-GOB-ES/roberta-base-bne"],  # Embeddings contextuales de entrenamiento
    datos_y_tarea2                                                            # Etiquetas numéricas correspondientes
)

# Conversión de embeddings contextuales a formato DMatrix para XGBoost
datos_entrenamiento_contextual = xgb.DMatrix(
    vectors["task2"]["spanish"]["train"]["PlanTL-GOB-ES/roberta-base-bne"],
    label=y_etiquetas_numericas2
)
datos_prueba_contextual = xgb.DMatrix(
    vectors["task2"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"],
    label=y_prueba_tarea2
)

# Parámetros de entrenamiento para XGBoost usando embeddings contextuales
params_contextual = {
    'max_depth': 3,              # Profundidad máxima de los árboles
    'eta': 0.1,                  # Tasa de aprendizaje
    'objective': 'multi:softmax',# Objetivo para clasificación multiclase
    'num_class': 3,              # Número de clases objetivo
    'eval_metric': 'mlogloss'    # Métrica de evaluación
}
num_rounds_contextual = 100      # Número de iteraciones del entrenamiento

# Entrenamiento del modelo XGBoost con embeddings contextuales
modelo_contextual_2 = xgb.train(
    params_contextual,
    datos_entrenamiento_contextual,
    num_rounds_contextual
)


C:\Users\marcos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [203]:
predicho1_1_estatico = clasificador_estatico_1.predict(lsa_vectores_prueba_tarea1)  

predicho1_1_contextual = clasificador_contextual_1.predict(vectors["task1"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"])  

# Generar predicciones en el conjunto de prueba  
predicho1_2_estatico = modelo_estatico_1.predict(datos_prueba_estatico)  
predicho1_2_estatico = np.asarray([int(round(valor)) for valor in predicho1_2_estatico])  

# Generar predicciones en el conjunto de prueba  
predicho1_2_contextual = modelo_contextual_1.predict(datos_prueba_contextual)  
predicho1_2_contextual = np.asarray([int(round(valor)) for valor in predicho1_2_contextual])  


# Generar predicciones para EnDevTask2 usando los modelos _2  
datos_prueba_tarea2 = xgb.DMatrix(lsa_vectores_prueba_tarea2, label=y_prueba_tarea2)  

predicho2_1_estatico = clasificador_estatico_2.predict(lsa_vectores_prueba_tarea2)  

predicho2_1_contextual = clasificador_contextual_2.predict(vectors["task2"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"])  

# Generar predicciones en el conjunto de prueba  
predicho2_2_estatico = modelo_estatico_2.predict(datos_prueba_tarea2)  
predicho2_2_estatico = np.asarray([int(round(valor)) for valor in predicho2_2_estatico])  

# Generar predicciones en el conjunto de prueba  
predicho2_2_contextual = modelo_contextual_2.predict(datos_prueba_contextual)  
predicho2_2_contextual = np.asarray([int(round(valor)) for valor in predicho2_2_contextual])  


In [ ]:
# Predicción utilizando clasificador estático (SVM) para la tarea 1
predicho1_1_estatico = clasificador_estatico_1.predict(lsa_vectores_prueba_tarea1)

# Predicción utilizando clasificador contextual (SVM) con embeddings de RoBERTa para la tarea 1
predicho1_1_contextual = clasificador_contextual_1.predict(
    vectors["task1"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"]
)

# Predicción utilizando modelo estático (XGBoost) para la tarea 1
predicho1_2_estatico = modelo_estatico_1.predict(datos_prueba_estatico)
# Conversión de predicciones a valores enteros
predicho1_2_estatico = np.asarray([int(round(valor)) for valor in predicho1_2_estatico])

# Predicción utilizando modelo contextual (XGBoost) con embeddings para la tarea 1
predicho1_2_contextual = modelo_contextual_1.predict(datos_prueba_contextual)
# Conversión de predicciones a valores enteros
predicho1_2_contextual = np.asarray([int(round(valor)) for valor in predicho1_2_contextual])

# Predicción utilizando clasificador estático (SVM) para la tarea 2
predicho2_1_estatico = clasificador_estatico_2.predict(lsa_vectores_prueba_tarea2)

# Predicción utilizando clasificador contextual (SVM) con embeddings de RoBERTa para la tarea 2
predicho2_1_contextual = clasificador_contextual_2.predict(
    vectors["task2"]["spanish"]["test"]["PlanTL-GOB-ES/roberta-base-bne"]
)

# Predicción utilizando modelo estático (XGBoost) para la tarea 2
predicho2_2_estatico = modelo_estatico_2.predict(datos_prueba_estatico)
# Conversión de predicciones a valores enteros
predicho2_2_estatico = np.asarray([int(round(valor)) for valor in predicho2_2_estatico])

# Predicción utilizando modelo contextual (XGBoost) con embeddings para la tarea 2
predicho2_2_contextual = modelo_contextual_2.predict(datos_prueba_contextual)
# Conversión de predicciones a valores enteros
predicho2_2_contextual = np.asarray([int(round(valor)) for valor in predicho2_2_contextual])

## Show Results

In [ ]:
from sklearn.metrics import f1_score, classification_report

# Función para mostrar resultados de la tarea 1 (clasificación binaria)
def mostrar_tarea1(referencia, prediccion):
    # Calcula F1 para la clase positiva
    f1_positivo = f1_score(referencia, prediccion, pos_label=1, zero_division=0)
    # Genera un reporte de clasificación detallado
    reporte = classification_report(referencia, prediccion, digits=4, zero_division=0)
    print("\nTAREA 1 - Resultados")
    print(f"Puntuación F1 (Clase Positiva): {f1_positivo:.4f}")
    print(reporte)

# Función para mostrar resultados de la tarea 2 (clasificación multiclase)
def mostrar_tarea2(referencia, prediccion):
    # Calcula F1 promedio macro (útil para clasificación multiclase)
    f1_macro = f1_score(referencia, prediccion, average='macro', zero_division=0)
    # Genera un reporte de clasificación detallado
    reporte = classification_report(referencia, prediccion, digits=4, zero_division=0)
    print("\nTAREA 2 - Resultados")
    print(f"Puntuación F1 (Promedio Macro): {f1_macro:.4f}")
    print(reporte)

# Imprime encabezado principal
print("==== RESULTADOS PARA ESPAÑOL ====\n")

# Resultados usando embebimientos estáticos
print("EMBEBIMIENTOS ESTÁTICOS\n")

print("TAREA 1\n=================================\n")
print("SVM: ")
mostrar_tarea1(y_prueba_tarea1, predicho1_1_estatico)

print("=================================\nXGBoost: ")
mostrar_tarea1(y_prueba_tarea1, predicho1_2_estatico)
print("=================================\n")

print("TAREA 2\n=================================\n")
print("SVM: ")
mostrar_tarea2(y_prueba_tarea2, predicho2_1_estatico)

print("=================================\nXGBoost: ")
mostrar_tarea2(y_prueba_tarea2, predicho2_2_estatico)
print("=================================\n")

# Resultados usando embebimientos contextuales
print("EMBEBIMIENTOS CONTEXTUALES\n")

print("TAREA 1\n=================================\n")
print("SVM: ")
mostrar_tarea1(y_prueba_tarea1, predicho1_1_contextual)

print("=================================\nXGBoost: ")
mostrar_tarea1(y_prueba_tarea1, predicho1_2_contextual)
print("=================================\n")

print("TAREA 2\n=================================\n")
print("SVM: ")
mostrar_tarea2(y_prueba_tarea2, predicho2_1_contextual)

print("=================================\nXGBoost: ")
mostrar_tarea2(y_prueba_tarea2, predicho2_2_contextual)
print("=================================\n")
